In [2]:
# import needed libraries
from pandas_profiling import ProfileReport
import pandas as pd
#import os
import wandb
#import hydra
#from omegaconf import DictConfig


In [5]:
# connect to weights and biases and create a run
run = wandb.init(job_type="notebook_eda", project="nyc_airbnb", group="eda", save_code=True)
# download the training data as csv file
local_path = wandb.use_artifact("sample.csv:latest").file()
# read it and provide it as pandas dataframe
df = pd.read_csv(local_path)

wandb: Currently logged in as: rainerenglisch (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [6]:
# create a profiling report
profile = ProfileReport(df)
# visualize profiling report
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

- Understand what each feature means
- Univariate analysis to verify that our expectation on that feature matches reality
- Bivariate analysis where we look for correlations
- Anomaly detection
- Missing values handling

In [7]:
# Drop outliers of price feature
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()
# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

In [10]:
# Transform object features to categorical features
df = pd.concat([
        df.select_dtypes([], ['object']),
        df.select_dtypes(['object']).apply(pd.Series.astype, dtype='category')
        ], axis=1)#.reindex_axis(df.columns, axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19001 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              19001 non-null  int64         
 1   host_id                         19001 non-null  int64         
 2   latitude                        19001 non-null  float64       
 3   longitude                       19001 non-null  float64       
 4   price                           19001 non-null  int64         
 5   minimum_nights                  19001 non-null  int64         
 6   number_of_reviews               19001 non-null  int64         
 7   last_review                     15243 non-null  datetime64[ns]
 8   reviews_per_month               15243 non-null  float64       
 9   calculated_host_listings_count  19001 non-null  int64         
 10  availability_365                19001 non-null  int64         
 11  na

In [ ]:
run.finish()